In [290]:
%load_ext autoreload
%autoreload 2
%env ANYWIDGET_HMR=1
%env ANYWIDGET_DEV=1

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
env: ANYWIDGET_HMR=1
env: ANYWIDGET_DEV=1


In [291]:
import networkx as nx
import numpy as np
from datasets import load_dataset
from numpy.random import choice

from seq import Widget

ds = load_dataset("ajaykarthick/imdb-movie-reviews")["test"]

In [373]:
w = Widget(
  sequences=[m["review"] for m in ds if m["label"] == 1],
  max_length=16,
  n_features=10,
)
w

Widget(labels=[{'id': 214, 'token': 'movie'}, {'id': 4, 'token': 'film'}, {'id': 30, 'token': 'one'}, {'id': 6…

In [374]:
n = 100

In [375]:
raw_sequences = w.sequences

In [376]:
w.sequences = raw_sequences[:n].copy()

In [377]:
org = np.array(raw_sequences[:n]).copy()
new_seq = np.array(raw_sequences[:n]).copy()

mask = np.isin(new_seq, np.array([i["id"] for i in w.labels]))
new_seq = new_seq * mask

ORG = np.array(org)
A = np.array(new_seq)
n_window = A.shape[1]

A_expanded = A[:, np.newaxis, :]
dist_matrix = (
  (A_expanded == A_expanded.transpose(1, 0, 2)) * (A_expanded != 0)
).sum(axis=2)
np.fill_diagonal(dist_matrix, 0)

G = nx.Graph()
for i in range(len(dist_matrix)):
  for j in range(i + 1, len(dist_matrix)):
    G.add_edge(i, j, weight=n_window - dist_matrix[i, j])


answer = nx.algorithms.approximation.christofides(G)
# answer = nx.algorithms.approximation.traveling_salesman_problem(G)

w.sequences = ORG[np.array(answer)].tolist()


def get_scores(matrix):
  return (((matrix[:-1, :] - matrix[1:, :]) == 0) * (matrix[:-1, :] != 0)).sum()


print(get_scores(ORG))
print(get_scores(ORG[np.array(answer)]))

2
63


In [378]:
unmasked = ORG[np.array(answer)].copy()

mask1 = unmasked[:-2, :] != unmasked[1:-1, :]
mask2 = unmasked[1:-1, :] != unmasked[2:, :]
mask = mask1 & mask2

unmasked[1:-1, :][mask] = 0

In [379]:
w.sequences = unmasked.tolist()